In [4]:
# importing libraries
import tweepy
from textblob import TextBlob
from wordcloud import WordCloud

import configparser
import os

import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [95]:
# login to twitter dev account
config = configparser.ConfigParser()
config.read('config.ini')



api_key = config['twitter']['api_key']
api_key_secret = config['twitter']['api_key_secret']

access_token = config['twitter']['access_token']
access_token_secret = config['twitter']['access_token_secret']


# aunthenticate

auth = tweepy.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)


api = tweepy.API(auth, wait_on_rate_limit=True)

## Data Gathering

In [15]:
## Reality Check! 
## Pull 200 HouseDemocrat tweets to test connection
posts = api.user_timeline(screen_name = "HouseDemocrats", count=200, tweet_mode="extended")


tweets = []
columns=['user','text','date','favs','retweets']
for tweet in posts:
    tweets.append([tweet.user.screen_name, tweet.full_text, tweet.created_at, tweet.favorite_count, tweet.retweet_count])

dftweets = pd.DataFrame(tweets, columns=columns)    
dftweets
    
  

,user,text,date,favs,retweets
0,HouseDemocrats,RT @RepJeffries: Heading back to Washington.\n...,2022-01-10 16:58:01+00:00,0,1258
1,HouseDemocrats,American infrastructure was last modernized ov...,2022-01-10 14:30:01+00:00,139,40
2,HouseDemocrats,RT @RepLoisFrankel: Sign up for ACA Marketplac...,2022-01-09 22:57:00+00:00,0,21
3,HouseDemocrats,RT @HouseBudgetDems: .@POTUS is right. The GOP...,2022-01-09 21:40:00+00:00,0,41
4,HouseDemocrats,RT @RepKirkpatrick: Good Employment News this ...,2022-01-09 18:53:00+00:00,0,20
...,...,...,...,...,...
195,HouseDemocrats,RT @WhiteHouse: Vaccine update: \n2.2 million ...,2021-12-03 22:54:00+00:00,0,2168
196,HouseDemocrats,BREAKING: Last month unemployment sunk⬇️ to th...,2021-12-03 22:18:00+00:00,198,60
197,HouseDemocrats,December dates to mark on your calendar 🗓. \n\...,2021-12-03 21:28:00+00:00,95,37
198,HouseDemocrats,RT @WhiteHouse: Economic update:\n- unemployme...,2021-12-03 20:00:19+00:00,0,2736


In [85]:
## functionalizing pull of more than 200 tweets using cursor

def get_hist_Tweets(user):
    
    tweets = []
    columns=['user','text','date','fav','rt']
        
    for tweet in tweepy.Cursor(api.user_timeline,screen_name=user).items(3200):
        tweets.append([tweet.user.screen_name, 
                       tweet.text, 
                       tweet.created_at, 
                       tweet.favorite_count, 
                       tweet.retweet_count])
    tempdf = pd.DataFrame(tweets, columns=columns)
    return tempdf
    

In [105]:
##tweets from Democrat Congresspeople and Senators
Democrat_Users = ['POTUS',
                  'SpeakerPelosi',
                  'KamalaHarris',                  
                  'VP',
                  'JoeBiden',
                  'RepAdamSchiff',
                  'RepMaxineWaters',
                  'IlhanMN',
                  'SenSchumer',
                  'SecretaryPete',
                  'USDOT',
                  'amyklobuchar',
                  'staceyabrams',
                  'Sen_JoeManchin',
                  'SenSanders',
                  'AOC',
                  'SenatorDurbin',
                  'DNC',
                  'HouseDemocrats',
                  'SenateDems',
                  'harrisonjaime',
                  'RepRaskin',
                  'OversightDems',
                  'RepMaloney',
                  'WhiteHouse',
                  'RashidaTlaib',
                  'CoriBush',
                  'PramilaJayapal']

##tweets from Republican Congresspeople and Senators
Republican_Users = ['Mike_Pence',
                    'LeaderMcConnell',
                    'LindseyGrahamSC',
                    'marcorubio',
                    'BillHagertyTN',
                    'Hawley_MO',
                    'GovRonDeSantis',
                    'RepCawthorn',
                    'Jim_Jordan',
                    'SenTedCruz',
                    'RepMattGaetz',
                    'RepBoebert',
                    'MarshaBlackburn',
                    'JohnCornyn',
                    'SenRonJohnson',
                    'lisamurkowski',
                    'SenatorCollins',
                    'RepLizCheney',
                    'MittRomney',
                    'RNC',
                    'RepGosar',
                    'replouiegohmert',
                    'MarkMeadows',
                    'GOPLeader',
                    'RandPaul',
                    'GOPoversight',
                    'GOP',
                    'HouseGOP',
                    'SenateGOP']

    
    

In [1]:
# get all tweets from listed Dems  
all_dem_tweets  = pd.DataFrame()#
for dem in Democrat_Users:
    temp_tweets = get_hist_Tweets(dem)
    all_dem_tweets = pd.concat([all_dem_tweets,temp_tweets])

  
    

In [ ]:
# get all tweets from listed Gop
all_gop_tweets  = pd.DataFrame()
for gop in Republican_Users:
    temp_tweets = get_hist_Tweets(gop)
    all_gop_tweets = pd.concat([all_gop_tweets,temp_tweets])


In [121]:
all_dem_tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80947 entries, 0 to 3199
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype              
---  ------  --------------  -----              
 0   user    80947 non-null  object             
 1   text    80947 non-null  object             
 2   date    80947 non-null  datetime64[ns, UTC]
 3   fav     80947 non-null  int64              
 4   rt      80947 non-null  int64              
dtypes: datetime64[ns, UTC](1), int64(2), object(2)
memory usage: 3.7+ MB


In [116]:
all_gop_tweets.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 75623 entries, 0 to 3199
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype              
---  ------  --------------  -----              
 0   user    75623 non-null  object             
 1   text    75623 non-null  object             
 2   date    75623 non-null  datetime64[ns, UTC]
 3   fav     75623 non-null  object             
 4   rt      75623 non-null  object             
dtypes: datetime64[ns, UTC](1), object(4)
memory usage: 3.5+ MB


## Exporting data as csv files

In [114]:
all_gop_tweets.to_csv('data/gop_tweets.csv', sep='\t', encoding='utf-8', index=False)
all_dem_tweets.to_csv('data/dem_tweets.csv', sep='\t', encoding='utf-8', index=False)

In [5]:
all_gop_tweets = pd.read_csv('data/gop_tweets.csv')

## Start Data Cleansing

In [2]:
import numpy as np
import re
 
def clean_tweet(tweet):
    if type(tweet) == np.float:
        return ""
    temp = tweet.lower()
    temp = re.sub("'", "", temp) # to avoid removing contractions in english
    temp = re.sub("@[A-Za-z0-9_]+","", temp)
    temp = re.sub("#[A-Za-z0-9_]+","", temp)
    temp = re.sub(r'http\S+', '', temp)
    temp = re.sub('[()!?]', ' ', temp)
    temp = re.sub('\[.*?\]',' ', temp)
    temp = re.sub("[^a-z0-9]"," ", temp)
    temp = temp.split()
    temp = [w for w in temp if not w in stopwords]
    temp = " ".join(word for word in temp)
    return temp

In [20]:
test_df = all_gop_tweets.head(20).copy()
#test_df.rename(columns={'text':'tweet_txt'}, inplace=True)

In [24]:
tweets_df=test_df.loc[:,['text']]
tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    20 non-null     object
dtypes: object(1)
memory usage: 288.0+ bytes


In [30]:
a = 9
for i in range(a,a+10):
    print(tweets_df.text[i])
    print()

RT @AmericanFreedom: https://t.co/8mElWdPaph https://t.co/Qul34SurDE

https://t.co/9thARayoUw

Two years ago today, US Forces carried out a drone strike killing the most dangerous terrorist in the world, Qasem… https://t.co/igGWAwuYOV

RT @AmericanFreedom: Pence group files SCOTUS brief opposing Biden vaccine mandate | @thehill  https://t.co/3f3rmXQDgn

And Happy Birthday To My Amazing Wife ⁦@KarenPence⁩! Her Love, Faith, Devotion to Our Family and Our Country Inspir… https://t.co/zXH944I760

Happy New Year America! Wishing You and Yours a Healthy and Prosperous 2022! 🇺🇸

RT @yaf: What. A. Year.

We brought conservative ideas to BILLIONS online and tens of THOUSANDS in-person on campuses in 2021.

And just wa…

RT @yaf: Faith and freedom go hand in hand. @Mike_Pence https://t.co/9Xnlw0gWst

RT @Mike_Pence: We Wish You A Merry Christmas! https://t.co/zQCP3sErTK

We Wish You A Merry Christmas! https://t.co/zQCP3sErTK



In [33]:
import re
import string
def clean_text(text):
    #''', , and '''
#Make text lowercase   
    text = text.lower()
#remove text in square brackets
    text = re.sub(r'\[.*?\]', '', text)
#remove punctuation   
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text) 
#remove words containing numbers
    text = re.sub(r'\w*\d\w*', '', text)
    return text

tweets_df_clean = pd.DataFrame(tweets_df.text.apply(lambda x: clean_text(x)))

In [34]:
tweets_df_clean

,text
0,us inflation hit in december reaches fastest ...
1,rt heritage senate democrats used to defend th...
2,rt senategop rampant covid a crumbling economy...
3,on this national law enforcement appreciation ...
4,rt senategop 🚨 jobs report 🚨\n\neconomists pre...
5,rt americanfreedom
6,rt americanfreedom start the year off right an...
7,former vp mike pence asks supreme court to blo...
8,rt americanfreedom joe biden’s vaccine mandate...
9,rt americanfreedom


In [37]:
!pip install spacy

In [43]:
import spacy
import en_core_web_sm

nlp = en_core_web_sm.load()
def lemmatizer(text):        
    sent = []
    doc = nlp(text)
    for word in doc:
        sent.append(word.lemma_)
    return " ".join(sent)
tweets_df_clean = pd.DataFrame(tweets_df_clean.text.apply(lambda x: lemmatizer(x)))
tweets_df_clean['text'] = tweets_df_clean['text'].str.replace('-PRON-', '')

In [44]:
tweets_df_clean

,text
0,we inflation hit in december reach fast pace...
1,rt heritage senate democrat use to defend the ...
2,rt senategop rampant covid a crumble economy c...
3,on this national law enforcement appreciation ...
4,rt senategop 🚨 job report 🚨 \n\n economist pre...
5,rt americanfreedom
6,rt americanfreedom start the year off right an...
7,former vp mike penny ask supreme court to bloc...
8,rt americanfreedom joe biden ’s vaccine mandat...
9,rt americanfreedom


In [46]:
## Unigrams
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
def get_top_n_words(corpus, n=None):
    vec = CountVectorizer(stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_words(tweets_df_clean.text, 10)
unigram = pd.DataFrame(common_words, columns = ['unigram' , 'count'])

In [47]:
unigram

,unigram,count
0,rt,12
1,americanfreedom,5
2,american,4
3,biden,4
4,year,4
5,mandate,3
6,wish,3
7,senategop,2
8,economy,2
9,job,2


In [56]:
## bigrams
def get_top_n_trigram(corpus, n=None):
    vec = CountVectorizer(stop_words='english',ngram_range=(2,2)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in      vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_trigram(tweets_df_clean.text, 10)
bigram = pd.DataFrame(common_words, columns = ['bigram' , 'count'])

In [57]:
bigram

,bigram,count
0,rt americanfreedom,5
1,rt senategop,2
2,biden vaccine,2
3,vaccine mandate,2
4,rt yaf,2
5,wish merry,2
6,merry christmas,2
7,inflation hit,1
8,hit december,1
9,december reach,1


In [54]:
## Trigrams
def get_top_n_trigram(corpus, n=None):
    vec = CountVectorizer(stop_words='english',ngram_range=(3,3)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in      vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_trigram(tweets_df_clean.text, 10)
trigram = pd.DataFrame(common_words, columns = ['trigram' , 'count'])

In [55]:
trigram

,trigram,count
0,biden vaccine mandate,2
1,wish merry christmas,2
2,inflation hit december,1
3,hit december reach,1
4,december reach fast,1
5,reach fast pace,1
6,fast pace wsj,1
7,rt heritage senate,1
8,heritage senate democrat,1
9,senate democrat use,1


## Topic Modeling

In [65]:
from sklearn.decomposition import LatentDirichletAllocation
vectorizer = CountVectorizer(
    analyzer='word',       
    min_df=3,# minimum required occurences of a word 
    stop_words='english',# remove stop words
    lowercase=True,# convert all words to lowercase
    token_pattern='[a-zA-Z0-9]{3,}',# num chars > 3
    max_features=5000 # max number of unique words
    )
data_matrix = vectorizer.fit_transform(tweets_df_clean.text)
data_matrix

<20x6 sparse matrix of type '<class 'numpy.int64'>'
	with 22 stored elements in Compressed Sparse Row format>

In [67]:
lda_model = LatentDirichletAllocation(
n_components=10, # Number of topics
learning_method='online',
random_state=20,       
n_jobs = -1  # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_matrix)

## using pyLDAvis

In [77]:
#!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_model, data_matrix, vectorizer, mds='tsne')

/Users/embernardo/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


TypeError: import_optional_dependency() got an unexpected keyword argument 'errors'

/Users/embernardo/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Requirement already up-to-date: pandas in /Users/embernardo/opt/anaconda3/lib/python3.8/site-packages (1.3.5)


In [15]:
results = [clean_tweet(tw) for tw in tweets]
results

NameError: name 'stopwords' is not defined